## データサイエンス特論2024: Lecture14 LLM時代のデータサイエンスとは
### LocalなLLMでcsvデータを分析してみる。

>【注意】この環境は2024.5月に動作確認したものであり、利用するソフトのバージョンアップで使えなくなる可能性が高いことに注意。

ここでは、LLM(大規模言語モデル）を使って、csvファイルに記録されたデータを分析する例を示します。最先端のLLMの活用方法として、データに直接質問(query)するパターンと、データ分析エージェント(Agent)を使って分析させるパターンの２種類を紹介します。

OpenAIなどの有料の高性能LLMを使わなくても、最近はオープンソースのLLMでもcsvデータ分析ぐらいはできるものが増えてきました。試してみると面白と思います。

最初に、オープンソースのLLMを利用する例を、最後に参考までに、OpenAIを使う場合を示します。


オープンソースLLMを利用する方法として、ここでは、いったんローカル環境(といってもcolaboratory計算機ですが）にLLMをダウンロードして利用するローカルLLMを利用します。情報が外部に漏れることを気にする方はローカルLLMは便利ですね。ここではOllamaというソフトウエアを用いて、オープンソースのLLMをローカル環境にダウンロードして活用します。

OpenAIのAPIアカウントをお持ちの方は、gpt3やgpt4をLLMとして使うと、違いが実感できるはずです。

llamaindexの pandas query engineを使います。

- https://docs.llamaindex.ai/en/stable/examples/query_engine/pandas_query_engine/?h=titanic#analyzing-the-titanic-dataset



## Ollamaの設定
ここでは、ollamaを colaboratoryの環境にダウンロードしてインストールします、インストールは１回でいいです。

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh
!sudo apt install -y neofetch

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  caca-utils chafa fonts-droid-fallback fonts-noto-mono fonts-urw-base35
  ghostscript gsfonts imagemagick imagemagick-6-common imagemagick-6.q16 jp2a
  libchafa0 libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9
  libgs9-common libid3tag0 libidn12 libijs-0.35 libimlib2 libjbig2dec0
  libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10 libp

マシンスペックを確認します。

In [2]:
!neofetch

            .-/+oossssoo+/-.
        `:+ssssssssssssssssss+:`
      -+ssssssssssssssssssyyssss+-
    .ossssssssssssssssssdMMMNysssso.
   /ssssssssssshdmmNNmmyNMMMMhssssss/
  +ssssssssshmydMMMMMMMNddddyssssssss+
 /sssssssshNMMMyhhyyyyhmNMMMNhssssssss/
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
ossyNMMMNyMMhsssssssssssssshmmmhssssssso
+sssshhhyNMMNyssssssssssssyNMMMysssssss+
.ssssssssdMMMNhsssssssssshNMMMdssssssss.
 /sssssssshNMMMyhhyyyyhdNMMMNhssssssss/
  +sssssssssdmydMMMMMMMMddddyssssssss+
   /ssssssssssshdmNNNNmyNMMMMhssssss/
    .ossssssssssssssssssdMMMNysssso.
      -+sssssssssssssssssyyyssss+-
        `:+ssssssssssssssssss+:`
            .-/+oossssoo+/-.
root@cdf24a46c31d 
----------------- 
OS: Ubuntu 22.04.3 LTS x86_64 
Host: Google Compute Engine 
Kernel: 6.1.58+ 
Uptime: 18 mins 
Packages: 1300 (dpkg) 
Shell: bash 5.1.16 
Terminal: jupyter-noteboo 
CPU: Intel Xeon (8) @ 2.299GHz 
GPU: NVIDIA Tesla 

### ollama serverを起動

colab環境にてollamaを使うためのサーバーを起動します。このサーバーは、外部から(pythonなどから）ollamaが管理するllmにアクセスるためのサーバーでもあります。

ローカルLLMを使っていて、時々以下のようなエラーが出る場合は、ollama severが停止している可能性があるので、↓のセルをもう一度起動しましょう。

> ConnectError: [Errno 99] Cannot assign requested address

In [70]:
import subprocess
import time

# Start ollama as a backrgound process
command = "nohup ollama serve&"

# Use subprocess.Popen to start the process in the background
process = subprocess.Popen(command,
                            shell=True,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)
print("Process ID:", process.pid)
time.sleep(5)  # Makes Python wait for 5 seconds

Process ID: 26784


### ここでollamaでモデルをダウンロードしてrunします。


- 途中で落ちると、ollamaのプロセスが落ちるだけなので、そこだけ再起動すればよい、HDDにpullした結果が残っている？ので、再起動すると、すでにpullしたものはそのまま使えます。

```
# これはコードとして書式設定されます
ollama run mistral
ollama run llama3
ollama run phi3
ollama run nomic-embed-text
```



とりあえず、mistralが優秀なのでこれを使います、他のllmも試すとよいと思います。

In [6]:
OLLAMA_MODEL='gemma:7b'
OLLAMA_MODEL='mistral'
#OLLAMA_MODEL='phi3'
#OLLAMA_MODEL='wizardlm2'
#OLLAMA_MODEL='llama3'
# 70bは是対動かない
#OLLAMA_MODEL='llama3:70b'

# Set it at the OS level
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL

mistral


以下では、"run"コマンドにより、指定したローカルLLMをダウンロードして、実行しています。一度ダウンロードすると、セッションが終了するまでずっと使えます。

In [7]:
!ollama run $OLLAMA_MODEL "Explain AI in one line"

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   0% ▕▏ 498 KB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   2% ▕▏  61 MB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   4% ▕▏ 172 MB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   5% ▕▏ 218 MB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   8% ▕▏ 331 MB/4.1 GB                  

In [8]:
!pip install --quiet -U langchain langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.0 MB/s eta 0:00:00


ちゃんと動くかどうかを、確かめます。

In [9]:
from langchain_community.chat_models import ChatOllama

#OLLAMA_MODEL='mistral'
llm = ChatOllama(model=OLLAMA_MODEL)

llm.invoke("Who are you?")

AIMessage(content=" I'm an artificial intelligence designed to assist and engage in conversations with users. I don't have the ability to have a personal identity or emotions. I'm here to help answer questions, provide information, and make interactions more enjoyable. How can I assist you today?", response_metadata={'model': 'mistral', 'created_at': '2024-05-05T06:22:43.763356451Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 1425266630, 'load_duration': 1218803, 'prompt_eval_count': 8, 'prompt_eval_duration': 101616000, 'eval_count': 57, 'eval_duration': 1197803000}, id='run-1260076e-ce18-4fd5-bfff-8467012fe325-0')

# ここから本番です。

In [10]:
!pip install llama-index llama-index-experimental llama-index-llms-ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00


## まずは、簡単なDataFrame に対して、質問(query)を投げるパターンでの利用です。
- llamaindexというLLMを活用するpython パッケージを利用しています。
- llamaindexから、バックグラウンドで動いている、Ollamaモデルに接続する形でLLMを利用できます。


In [11]:
import logging
import sys
from IPython.display import Markdown, display

import pandas as pd
from llama_index.experimental.query_engine import PandasQueryEngine


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [71]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama

Settings.llm = Ollama(model=OLLAMA_MODEL, request_timeout=30.0)

もっとも簡単な例で、DataFrameに対する質問をしてみます。

In [72]:
# Test on some sample data
df = pd.DataFrame(
    {
        "city": ["Toronto", "Tokyo", "Berlin"],
        "population": [2930000, 13960000, 3645000],
    }
)

In [73]:
query_engine = PandasQueryEngine(df=df, verbose=True)

In [74]:
response = query_engine.query(
    "What is the name of the city with the highest population?",
)

> Pandas Instructions:
```
 df['city'][df['population'].idxmax()]
```
> Pandas Output: Tokyo


## ここから、いよいよtitanicに

In [75]:
!wget 'https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/docs/examples/data/csv/titanic_train.csv' -O 'titanic_train.csv'

--2024-05-05 07:41:22--  https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/docs/examples/data/csv/titanic_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57726 (56K) [text/plain]
Saving to: ‘titanic_train.csv’

titanic_train.csv   100%[===================>]  56.37K  --.-KB/s    in 0.01s   

2024-05-05 07:41:22 (5.23 MB/s) - ‘titanic_train.csv’ saved [57726/57726]



In [76]:
df = pd.read_csv("./titanic_train.csv" )
df.head()

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [77]:
query_engine = PandasQueryEngine(df=df, verbose=True)

In [78]:
response = query_engine.query(
    "What is the correlation between survival and age?",
)

> Pandas Instructions:
```
 df['survived'].corr(df['age'])
```
> Pandas Output: -0.07722109457217768


In [90]:
response = query_engine.query(
    "whats the square root of the average age?" ,
)

> Pandas Instructions:
```
 import numpy as np
np.sqrt(df['age'].mean())
```
> Pandas Output: There was an error running the output as Python code. Error message: Execution of code containing references to private or dunder methods, disallowed builtins, or any imports, is forbidden!


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/llama_index/experimental/query_engine/pandas/output_parser.py", line 42, in default_output_processor
    safe_exec(ast.unparse(module), {}, local_vars)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/llama_index/experimental/exec_utils.py", line 170, in safe_exec
    _verify_source_safety(__source)
  File "/usr/local/lib/python3.10/dist-packages/llama_index/experimental/exec_utils.py", line 144, in _verify_source_safety
    raise RuntimeError(
RuntimeError: Execution of code containing references to private or dunder methods, disallowed builtins, or any imports, is forbidden!


In [95]:
#agent.run('what is the most survived group? what is the opposite?')
response = query_engine.query(
    "what is the most survived group? what is the opposite?",
)

ReadTimeout: timed out

## agentでやってみる。

In [22]:
!pip install langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 4.2 MB/s eta 0:00:00


In [62]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_community.chat_models import ChatOllama

agent = create_pandas_dataframe_agent( ChatOllama(model=OLLAMA_MODEL), df, verbose=True)

In [63]:
agent.run("give me explanation for each column")



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: ` Thought: I will use the `describe` function of pandas to get summary statistics and column information for each column.
Action: python_repl_ast "df.describe()"`

In [64]:
agent.run("whats the square root of the average age?")



> Entering new AgentExecutor chain...
 Thought: To find the square root of the average age, I first need to calculate the average age using the mean() function from pandas.
Action: python_repl_ast
Action Input: df['age'].mean()29.69911764705882 Now that I have the average age, I can find its square root using Python's math.sqrt() function.

Action: python_repl_ast
Action Input: import math; math.sqrt(df['age'].mean())5.449689683556195 Final Answer: The square root of the average age is approximately 5.45.

> Finished chain.


'The square root of the average age is approximately 5.45.'

In [25]:
agent.run('what is the most survived group? what is the opposite?')



> Entering new AgentExecutor chain...
 Thought: To find out which group has the most survivors, I need to count the number of surviving passengers for each group (based on pclass). I can use pandas to do this.

Action: df.groupby('pclass')['survived'].sum()
Action Input:df.groupby('pclass')['survived'].sum() is not a valid tool, try one of [python_repl_ast]. I must use the pandas dataframe to find the answer. I'll first filter the dataframe for survived passengers, then group by 'pclass', and finally count the number of rows in each group.

Action: df[df['survived'] == 1].groupby('pclass').size().index
Action Input: [3, 1]
Thought: The numbers [3, 1] represent the indices of the groups with the most and least survivors, respectively. I need to look up these indices in the original dataframe to get the actual group labels.

Action: df.loc[df['pclass'].isin([3, 1])]['pclass']
Action Input:df[df['survived'] == 1].groupby('pclass').size().index is not a valid tool, try one of [python_rep

'The first class (pclass=1) has the most survivors, and the third class (pclass=3) has the least.'

# ご参考：OpenAIでやる場合

【注意】OpenAIのアカウントをもち、API_KEYを取得済みの人は、↓のセルでは、以下のようにやってみてください。

```
os.environ["OPENAI_API_KEY"] = XXXXXXX
openai.api_key = os.environ["OPENAI_API_KEY"]
```



In [32]:
#OpenAIの場合
import os
import openai
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key = os.environ["OPENAI_API_KEY"]

In [34]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

Settings.llm =  OpenAI(temperature=0.1,  model="gpt-3.5-turbo-instruct")

In [35]:
df.head()

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [36]:
query_engine = PandasQueryEngine(df=df, verbose=True)

In [39]:
response = query_engine.query(
    "What is the correlation between survival and age?",
)

> Pandas Instructions:
```
 df['survived'].corr(df['age'])
```
> Pandas Output: -0.07722109457217768


In [38]:
response = query_engine.query(
    "what is the most survived group? what is the opposite?",
)

Retrying llama_index.llms.openai.base.OpenAI._complete in 0.07634705626154525 seconds as it raised InternalServerError: Error code: 500 - {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_ac3f8cc69420763795a147c3c3d8df22 in your email.)', 'type': 'server_error', 'param': None, 'code': None}}.
> Pandas Instructions:
```
 df['survived'].value_counts().idxmax()
```
> Pandas Output: 0


In [41]:
!pip install langchain_openai

In [42]:
from langchain_openai import OpenAI

agent = create_pandas_dataframe_agent( OpenAI(temperature=0), df, verbose=True)

In [44]:
agent.run('what is the most survived group? what is the opposite?')



> Entering new AgentExecutor chain...
Thought: I need to find the group with the highest number of survivors and the group with the lowest number of survivors.
Action: python_repl_ast
Action Input: df['survived'].value_counts()survived
0    549
1    342
Name: count, dtype: int64549 people did not survive and 342 people did survive.
Action: python_repl_ast
Action Input: df['pclass'].value_counts()pclass
3    491
1    216
2    184
Name: count, dtype: int64491 people were in the third class, 216 people were in the first class, and 184 people were in the second class.
Action: python_repl_ast
Action Input: df.groupby('pclass')['survived'].sum()pclass
1    136
2     87
3    119
Name: survived, dtype: int64136 people in the first class survived, 87 people in the second class survived, and 119 people in the third class survived.
Action: python_repl_ast
Action Input: df.groupby('pclass')['survived'].sum().idxmax()11 is the index of the group with the highest number of survivors, which is the 

'The most survived group is the first class and the opposite is the second class.'

In [46]:
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [47]:
agent.run('what is the most survived group? what is the opposite?')



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['survived'].value_counts()"}`


survived
0    549
1    342
Name: count, dtype: int64The most survived group is the one with a value of 1, which indicates that the person survived. There are 342 individuals in this group.

The opposite group is the one with a value of 0, which indicates that the person did not survive. There are 549 individuals in this group.

> Finished chain.


'The most survived group is the one with a value of 1, which indicates that the person survived. There are 342 individuals in this group.\n\nThe opposite group is the one with a value of 0, which indicates that the person did not survive. There are 549 individuals in this group.'

In [48]:
agent.run("what is the most survived group? what is the opposite? igonore 'survived' ")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.groupby(['pclass', 'sex']).size().idxmax()"}`


(3, 'male')
Invoking: `python_repl_ast` with `{'query': "df.groupby(['pclass', 'sex']).size().idxmin()"}`


(2, 'female')The most survived group is the group of males in the third class (pclass=3). The opposite group, with the lowest survival rate, is the group of females in the second class (pclass=2).

> Finished chain.


'The most survived group is the group of males in the third class (pclass=3). The opposite group, with the lowest survival rate, is the group of females in the second class (pclass=2).'

In [50]:
agent.invoke("whats the square root of the average age?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\nimport math\n\n# Create a dataframe\ndata = {'age': [22, 38, 26, 35, 35]}\ndf = pd.DataFrame(data)\n\n# Calculate the average age\naverage_age = df['age'].mean()\n\n# Calculate the square root of the average age\nsquare_root = math.sqrt(average_age)\n\nsquare_root"}`


5.585696017507576The square root of the average age is approximately 5.59.

> Finished chain.


{'input': 'whats the square root of the average age?',
 'output': 'The square root of the average age is approximately 5.59.'}